In [1]:
#Imports

import cv2
import numpy as np
import os
import pickle
import time
from datetime import datetime
import csv
from collections import defaultdict
from tensorflow.keras.models import load_model


In [2]:
#Load Label Map Function

def load_label_map(label_map_path):
    with open(label_map_path, 'rb') as f:
        label_map = pickle.load(f)
    return label_map


In [3]:
#Load CNN Model Function

def load_cnn_model(model_path):
    model = load_model(model_path)
    return model


In [4]:
#Parse ID and Name from Folder Function

def parse_id_and_name_from_folder(folder_name):
    id_name_parts = folder_name.split('-')
    if len(id_name_parts) != 2:
        raise ValueError(f"Invalid folder name format: {folder_name}")
    person_id = id_name_parts[0]
    person_name = id_name_parts[1]
    return person_id, person_name


In [5]:
#Record Attendance Function

def record_attendance(attendance_records, date):
    COL_NAMES = ['ID', 'NAME', 'TIMESTAMP']
    file_path = f"Attendance/Attendance_{date}.csv"
    exist = os.path.isfile(file_path)

    with open(file_path, "a", newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not exist:
            writer.writerow(COL_NAMES)
        for record in attendance_records:
            writer.writerow(record)


In [6]:
#Get Most Confident Name Function

def get_most_confident_name(confidence_dict):
    most_confident_name = None
    highest_confidence = 0

    for name, confidences in confidence_dict.items():
        avg_confidence = np.mean(confidences)
        if avg_confidence > highest_confidence:
            highest_confidence = avg_confidence
            most_confident_name = name

    return most_confident_name, highest_confidence


In [7]:
#Main Function

def main():
    dataset_path = r'C:\Users\Sneha\Desktop\FACE RECOGNITION SYSTEM\data'
    label_map_path = 'models/label_map.pkl'
    model_path = 'models/cnn_face_recognizer_best_model.h5'

    label_map = load_label_map(label_map_path)
    print("Loaded label map:")
    print(label_map)

    model = load_cnn_model(model_path)

    video = cv2.VideoCapture(0)
    if not video.isOpened():
        print("Error: Could not open video capture.")
        return

    facedetect = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    start_time = time.time()
    date = datetime.now().strftime("%d-%m-%Y")

    attendance_records = []

    while True:
        ret, frame = video.read()
        if not ret:
            print("Error: Failed to capture frame.")
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = facedetect.detectMultiScale(gray, 1.3, 5)

        detected_persons = defaultdict(lambda: defaultdict(list))

        for (x, y, w, h) in faces:
            crop_img = gray[y:y + h, x:x + w]
            resized_img = cv2.resize(crop_img, (128, 128))
            img = np.expand_dims(resized_img, axis=0)
            img = np.expand_dims(img, axis=-1)
            img = img / 255.0

            pred = model.predict(img)
            label_index = np.argmax(pred)
            person_name = label_map[label_index]
            confidence = pred[0][label_index]

            try:
                person_id, person_name = parse_id_and_name_from_folder(person_name)
                detected_persons[(x, y, w, h)][f"{person_id}-{person_name}"].append(confidence)
            except ValueError as e:
                print(e)

            cv2.putText(frame, f'{person_name} ({confidence:.2f})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow("Attendance System", frame)

        current_time = time.time()
        if current_time - start_time >= 10:
            for (x, y, w, h), names_confidences in detected_persons.items():
                most_confident_name, highest_confidence = get_most_confident_name(names_confidences)
                try:
                    person_id, person_name = parse_id_and_name_from_folder(most_confident_name)
                    attendance_records.append([person_id, person_name, datetime.now().strftime("%H:%M:%S")])
                except ValueError as e:
                    print(e)
            record_attendance(attendance_records, date)
            break

        k = cv2.waitKey(1)
        if k == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()


In [8]:
#Execute Main Function

if __name__ == "__main__":
    main()


Loaded label map:
{0: '101-Aishwarya_Rai', 1: '102-Angelina_Jolie', 2: '103-Dhoni', 3: '104-Manjula', 4: '105-Sachin_Tendulkar', 5: '106-Saravanan', 6: '107-Sneha', 7: '108-Virat_Kohli'}


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━